### Riddler Express

Help, there’s a cricket on my floor! I want to trap it with a cup so that I can safely move it outside. But every time I get close, it hops exactly 1 foot in a random direction.

I take note of its starting position and come closer. Boom — it hops in a random direction. I get close again. Boom — it takes another hop in a random direction, independent of the direction of the first hop.

What is the most probable distance between the cricket’s current position after two random jumps and its starting position? (Note: This puzzle is not asking for the expected distance, but rather the most probable distance. In other words, if you consider the probability distribution over all possible distances, where is the peak of this distribution?)

In [1]:
import random
import math

In [2]:
degree = random.randrange(0.0,361)
print(f"Degrees of {degree} convert to radians of {math.radians(degree)}")

Degrees of 122 convert to radians of 2.129301687433082


In [3]:
degree = 180
rad = math.radians(degree)
x_val = math.cos(rad)
y_val = math.sin(rad)
print(f"Radian of {rad} has x-val of {x_val:.3f} and y-val of {y_val:.3f}")

Radian of 3.141592653589793 has x-val of -1.000 and y-val of 0.000


In [4]:
degree = 90
rad = math.radians(degree)
x_val = math.cos(rad)
y_val = math.sin(rad)
print(f"Radian of {rad} has x-val of {x_val:.3f} and y-val of {y_val:.3f}")

Radian of 1.5707963267948966 has x-val of 0.000 and y-val of 1.000


### Building A Class

Not necessary, but fun and get to use the default factory here. 

In [5]:
from dataclasses import dataclass, field

In [6]:

@dataclass(frozen=False) # we want immutability 
class cricket:
    coord: list = field(default_factory=lambda: [0, 0])
    degree: int = 180
        
    def random_jump(self):
        self.degree = random.randrange(0.0,361.0)
    
    def update_location(self):
        """Random degree to update coordinate"""
        rad = math.radians(self.degree)
        x_val = round(math.cos(rad),2)
        y_val = round(math.sin(rad),2)

        self.coord[0] += x_val
        self.coord[1] += y_val
    
    def get_distance(self):
        """Return euclidean distance from start"""
        a = (self.coord[0])**2
        b = (self.coord[1])**2
        return round((a + b) ** 0.5,2)
    
    def run(self):
        """Run sequence"""
        self.random_jump()
        self.update_location()
        self.random_jump()
        self.update_location()
        return self.get_distance()
        
    def __str__(self):
        return f"Cricket is at position {self.coord}" 

In [7]:
my_cricket = cricket()
print(my_cricket)
my_cricket.random_jump()
print(my_cricket.degree)
my_cricket.update_location()
print(my_cricket)
my_cricket.random_jump()
print(my_cricket.degree)
my_cricket.update_location()
print(my_cricket)
my_cricket.get_distance()

Cricket is at position [0, 0]
217
Cricket is at position [-0.8, -0.6]
151
Cricket is at position [-1.67, -0.12]


1.67

In [8]:
my_cricket = cricket()
my_cricket.run()

1.13

In [9]:
print(my_cricket)

Cricket is at position [-0.54, 0.99]


### Running A Simple Simulation

Over a variety of distances

In [10]:
from collections import defaultdict
from collections import Counter
import time

start = time.time()
sim_output = defaultdict(list) # store results from each sim
sim_size = [100, 1_000, 10_000, 100_000, 500_000, 1_000_000, 2_000_000]

for sim in sim_size:
    for _ in range(sim):
        my_cricket = cricket()
        sim_output[sim].append(my_cricket.run())
end = time.time()
print(f"Total time: {end - start:.3f} seconds")

Total time: 12.421 seconds


In [11]:
for sim in sim_size:
    data = Counter(sim_output[sim])
    print(f"Top 3 most common: {data.most_common(3)}")

Top 3 most common: [(1.75, 3), (1.78, 3), (1.74, 3)]
Top 3 most common: [(1.99, 47), (2.0, 38), (1.98, 29)]
Top 3 most common: [(1.99, 412), (2.0, 341), (1.98, 252)]
Top 3 most common: [(1.99, 3837), (2.0, 3604), (1.98, 2415)]
Top 3 most common: [(1.99, 19048), (2.0, 18147), (1.98, 11835)]
Top 3 most common: [(1.99, 38058), (2.0, 36567), (1.98, 23640)]
Top 3 most common: [(1.99, 76328), (2.0, 73390), (1.98, 47533)]


### That Was Super Slow Though - We Can Speed Through In Numpy

Since each operation is independent we see a massive speed-up using numpy (and this was a first pass, I bet we could make it EVEN FASTER!)

In [12]:
import numpy as np

start2 = time.time()
sim_output_np = {} # store results from each sim
sim_size = [100, 1_000, 10_000, 100_000, 500_000, 1_000_000, 2_000_000]

for sim in sim_size:
    # randomly determine radians
    arr = np.random.uniform(0, high=math.pi, size=sim)
    arr2 = np.random.uniform(0, high=math.pi, size=sim)

    # use radians to calculate total movement in x & y dir
    x_arr = np.cos(arr) + np.cos(arr2)
    y_arr = np.sin(arr) + np.sin(arr2)

    # euclidean distance
    sim_output_np[sim] = np.round((x_arr**2 + y_arr**2)**0.5,2)
    
end2 = time.time()
print(f"Total time: {end2 - start2:.3f} seconds")

Total time: 0.168 seconds


In [14]:
print(f"Numpy is {(end - start) / (end2 - start2):.0f} times faster than the class approach")

Numpy is 74 times faster than the class approach


In [15]:
for sim in sim_size:
    as_list = list(sim_output_np[sim])
    data = Counter(as_list)
    print(f"Top 3 most common for sim of {sim}: {data.most_common(3)}")

Top 3 most common for sim of 100: [(2.0, 9), (1.96, 7), (1.95, 5)]
Top 3 most common for sim of 1000: [(2.0, 94), (1.99, 72), (1.96, 36)]
Top 3 most common for sim of 10000: [(2.0, 824), (1.99, 593), (1.98, 426)]
Top 3 most common for sim of 100000: [(2.0, 8809), (1.99, 6290), (1.98, 4073)]
Top 3 most common for sim of 500000: [(2.0, 44311), (1.99, 30823), (1.98, 20815)]
Top 3 most common for sim of 1000000: [(2.0, 88163), (1.99, 62098), (1.98, 41490)]
Top 3 most common for sim of 2000000: [(2.0, 176526), (1.99, 123545), (1.98, 82703)]


### Final Solution

The most likely solution is 2.0